In [ ]:
import pandas as pd
#untuk membagi data menjadi data training dan data testing
from sklearn.model_selection import train_test_split
#ketika ada data kateogrical(y/n)
from sklearn.preprocessing import LabelEncoder
#untuk modeling
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
#untuk mencari feature importance metode knn
from sklearn.feature_selection import mutual_info_classif
#menggunakan xgboost untuk referensi melihat akurasi lebih bagus
import xgboost as xgb
# untuk menghitung true false & true negatif secara otomatis
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# Data Preprocessing

In [ ]:
# 1. load data train
df = pd.read_csv('/content/titanic-train.csv')
df1 = pd.DataFrame(df)
df1.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
# 2. Bersihkan data
# a. melihat jumlah kolom yang terdapat missing value
df1.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [ ]:
# b. mengisi kolom yang null pada kolom Age dengan median
df1['Age'].fillna(df1['Age'].median(), inplace=True)
df2 = df1.copy()
df2

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,28.0,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [ ]:
# c. menghapus beberapa kolom dan set PassengerId menjadi indeks
df3 = df2.drop(['Name','Sex','Ticket','Cabin','Embarked'], axis=1)
df3.set_index("PassengerId", inplace=True)

In [ ]:
df3

,Survived,Pclass,Age,SibSp,Parch,Fare
PassengerId,,,,,,
1,0,3,22.0,1,0,7.2500
2,1,1,38.0,1,0,71.2833
3,1,3,26.0,0,0,7.9250
4,1,1,35.0,1,0,53.1000
5,0,3,35.0,0,0,8.0500
...,...,...,...,...,...,...
887,0,2,27.0,0,0,13.0000
888,1,1,19.0,0,0,30.0000
889,0,3,28.0,1,2,23.4500


In [ ]:
# 3. Mengasumsikan Target Kolom
x = df3.drop('Survived', axis = 1)
y = df3['Survived']#disini saya menggunakan variabel Survived karena variabel ini lebih mudah melihat labelnya yaitu 1 = Surivive dan juga 0 = Tidak Survive

In [ ]:
x

,Pclass,Age,SibSp,Parch,Fare
PassengerId,,,,,
1,3,22.0,1,0,7.2500
2,1,38.0,1,0,71.2833
3,3,26.0,0,0,7.9250
4,1,35.0,1,0,53.1000
5,3,35.0,0,0,8.0500
...,...,...,...,...,...
887,2,27.0,0,0,13.0000
888,1,19.0,0,0,30.0000
889,3,28.0,1,2,23.4500


In [ ]:
y

PassengerId
1      0
2      1
3      1
4      1
5      0
      ..
887    0
888    1
889    0
890    1
891    0
Name: Survived, Length: 891, dtype: int64

# Pembuatan Model

In [ ]:

# pemisahan training dan testing
# fungsi train_test_split dari sklearn dengan test_size semakin kecil semakin bagus, ketika terlalu kecil predictnya jadi lebih jelek
# random state sebagai pengunci untuk menjaga konsistensi data agar ketika dirun dilain waktu tidak berubah (data test dan data traningnya)
# 4. Melakukan split data untuk mendapatkan data random yang digunakan dalam melatih model yang berisi data train dan data testing
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=123) # disini saya mengalokasikan 30% dari data train sebagai data uji dan 70%nya digunakan untuk melatih model

**Data Testing**

In [ ]:
x_test

,Pclass,Age,SibSp,Parch,Fare
PassengerId,,,,,
173,3,1.0,1,1,11.1333
525,3,28.0,0,0,7.2292
453,1,30.0,0,0,27.7500
171,1,61.0,0,0,33.5000
621,3,27.0,1,0,14.4542
...,...,...,...,...,...
373,3,19.0,0,0,8.0500
120,3,2.0,4,2,31.2750
418,2,18.0,0,2,13.0000


In [ ]:
y_test

PassengerId
173    1
525    0
453    0
171    0
621    0
      ..
373    0
120    0
418    1
585    0
592    1
Name: Survived, Length: 268, dtype: int64

**Data Training**

In [ ]:
x_train

,Pclass,Age,SibSp,Parch,Fare
PassengerId,,,,,
417,2,34.0,1,1,32.5000
802,2,31.0,1,1,26.2500
513,1,36.0,0,0,26.2875
456,3,29.0,0,0,7.8958
758,2,18.0,0,0,11.5000
...,...,...,...,...,...
99,2,34.0,0,1,23.0000
323,2,30.0,0,0,12.3500
383,3,32.0,0,0,7.9250


In [ ]:
y_train

PassengerId
417    1
802    1
513    1
456    1
758    0
      ..
99     1
323    1
383    0
366    0
511    1
Name: Survived, Length: 623, dtype: int64

**Modeling**

In [ ]:
# 5.Membuat Model dan Prediksi
# a. K-Nearest Neighbors (KNN)
knn = KNeighborsClassifier(n_neighbors=29) # disini saya menentukan n_neigbhor dengan cara melakukan pengakaran pada banyaknya rows pada data train sehingga diperoleh n_neighbhor = 29 atau 31, disini saya menggunakan 29 karena akurasinya lebih besar
knn.fit(x_train, y_train)
knn_pred = knn.predict(x_test)

# b. Decision Tree
dt = DecisionTreeClassifier()
dt.fit(x_train, y_train)
dt_pred = dt.predict(x_test)

# c. Random Forest
rf = RandomForestClassifier()
rf.fit(x_train, y_train)
rf_pred = rf.predict(x_test)

# d. XGBoost
xgb_model = xgb.XGBClassifier()
xgb_model.fit(x_train, y_train)
xgb_pred = xgb_model.predict(x_test)

# Evaluasi Model

In [ ]:
# Actual target values
actual = y_test
# 6. membuat confusion matrices 2 (variabel yang indeksnya sama)
knn_cm = confusion_matrix(actual, knn_pred)
dt_cm = confusion_matrix(actual, dt_pred)
rf_cm = confusion_matrix(actual, rf_pred)
xgb_cm = confusion_matrix(actual, xgb_pred)

In [ ]:
# 7. convert confusion matrics ke DataFrames
def confusion_matrix_to_dataframe(cm):
    labels = sorted(set(actual))
    df_cm = pd.DataFrame(cm, index=labels, columns=labels)
    df_cm.index.name = 'Actual'
    df_cm.columns.name = 'Predicted'
    return df_cm

In [ ]:
# 8. konversi confusion matrices ke ADataFrames dengan labels
knn_cm_df = confusion_matrix_to_dataframe(knn_cm)
dt_cm_df = confusion_matrix_to_dataframe(dt_cm)
rf_cm_df = confusion_matrix_to_dataframe(rf_cm)
xgb_cm_df = confusion_matrix_to_dataframe(xgb_cm)

In [ ]:
#melihat prediksi dengan metode knn
knn_cm_df

Predicted,0,1
Actual,,
0,142,28
1,46,52


In [ ]:

#melihat prediksi dengan decision tree
dt_cm_df

Predicted,0,1
Actual,,
0,112,58
1,45,53


In [ ]:
#melihatprediksi dengan  randomm forest
rf_cm_df

Predicted,0,1
Actual,,
0,125,45
1,41,57


In [ ]:
#melihat prediksi dengan xgboost
xgb_cm_df

Predicted,0,1
Actual,,
0,121,49
1,39,59


# Mengukur Akurasi Model

In [ ]:
# 9. Membuat fungsi unuk mengukur akurasi model
def evaluate(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    return cm, accuracy, precision, recall, f1

In [ ]:
# a. Evaluasi Akurasi KNN
knn_cm, knn_accuracy, knn_precision, knn_recall, knn_f1 = evaluate(y_test, knn_pred)
knn_f1

0.7168610528426227

In [ ]:
# b. Evaluasi Akurasi Tree
dt_cm, dt_accuracy, dt_precision, dt_recall, dt_f1 = evaluate(y_test, dt_pred)
dt_f1

0.6199848830995368

In [ ]:
# c. Evaluasi Akurasi Random Forest
rf_cm, rf_accuracy, rf_precision, rf_recall, rf_f1 = evaluate(y_test, rf_pred)
rf_f1

0.680403340440654

In [ ]:
# d. Evaluasi Akurasi XGBoost
xgb_cm, xgb_accuracy, xgb_precision, xgb_recall, xgb_f1 = evaluate(y_test, xgb_pred)
xgb_f1

0.6746365261073275

# Feature Importance

In [ ]:
# 10. Membuat Feature Importance
# saya menggunakan Knn karena akurasinya paling tinggi dengan menggunakan sklearn.feature_selection import mutual_info_classif karena tidak ada fitur importance di knn
# a. Menghitung mutual information antara fitur dan target
mutual_info = mutual_info_classif(x, y)

# b. Membuat DataFrame untuk menyimpan hasil
feature_names = x.columns
mi_df = pd.DataFrame({'Feature': feature_names, 'Mutual_Information': mutual_info})

# c. Mengurutkan hasil berdasarkan nilai mutual information
mi_df = mi_df.sort_values(by='Mutual_Information', ascending=False)
mi_df

,Feature,Mutual_Information
4,Fare,0.133726
0,Pclass,0.039231
1,Age,0.032905
2,SibSp,0.014271
3,Parch,0.000000


#**Penjelasan**

**Evaluasi model**

Berdasarkan hasil prediksi diatas ditemukan bahwa model KNN lebih cocok, dikarenakan hasil prediksi dari knn berdasarkan F1-Score memiliki Nilai True Negatif sebesar 142 dan True Positif 52, dalam kasus ini model KNN memiliki jumlah True Positif dan True Negatif yang paling besar diantara 4 model lainnya.
Dari data titanic ini untuk melihat tingkat keselamatan ( Survived ) agar lebih akurat dalam memprediksinya dilakukanlah pengukuran akurasi dan menghasilkan model yang memiliki akurasi tertinggi yaitu model KNN dengan tingkat akurasi sebesar 71.6%


**Variabel yang berpengaruh**

Untuk melihat variabel yang berpengaruh saya buat Feature Importance
  saya menggunakan Knn karena akurasinya paling tinggi dengan menggunakan sklearn.feature_selection import mutual_info_classif karena tidak ada fitur importance di knn
  langkahnya sebagai berikut

  a. menghitung information antara fitur dan target  
  b. membuat dataframe untuk menyimpan hasil

  c. mengurutkan hasil berdasarkan nilai mutual information didapatkan variabel yang berpengaruh yaitu Fare

  saya mencoba melihat feature importance menggunakan semua model, namun 3 model(KNN, Decision Tree, Random Forest) menunjukan variabel yang sangat berpengaruh yaitu Fare, dan model XGBoost menunjukan variabel yang berpengaruh yaitu Pclass
  dari percobaan feature importance diatas perbandingan antara variabel Fare dan Pclass yaitu 3:1
  sehingga dapat diambil keputusan variabel Fare lebih berpengaruh, beberapa alasannya yaitu

  a.  Penumpang yang membayar lebih mahal mungkin ditempatkan dikabin yang lebih tinggi atau dekat dengan area evakuasi, yang dapat meningkatkan peluang untuk selamat ( Survived )
  
  b. Selama evakuasi, kemungkinan bahwa penumpang yang membayar lebih mahal diberi prioritas dalam proses evakuasi

   c. Penumpang yang membayar lebih mahal akan mendapatkan kelas tiket lebih tinggi. Kelas yang lebih tinggi memiliki akses terbaik ke cabin / dek atas dan akses peralatan keselamatan tambahan  yang mempengaruhi tingkat keselamatan

   d. Penumpang dengan kelas tiket lebih tinggi sering kali memiliki status sosial yang lebih tinggi dan mungkin mendapatkan perhatian atau bantuan lebih selama keadaan darurat

# Melatih Model Pada Data titanic-test

In [ ]:
# 11. Load data titanic-test untuk diprediksi
df4 = pd.read_csv('/content/titanic-test.csv')
df4

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [ ]:
# 12. Bersihkan Data
# a. Melihat kolom yang null missing value dan jumlah missing value dalam sebuah variabel
df4.isna().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [ ]:
# b. mengisi kolom yang memiliki null value
df4['Age'].fillna(df1['Age'].median(), inplace=True) # mengisi null value pada variabel age dengan median
df4['Fare'].fillna(df1['Fare'].mean(), inplace=True) # mengisi null value pada variabel Fare dengan mean
df4.copy()
df4

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,28.0,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,28.0,0,0,359309,8.0500,NaN,S


In [ ]:
# c. menghapus kolom
df5 = df4.drop(['Name','Sex','Ticket','Cabin','Embarked'], axis=1)
df5.set_index("PassengerId", inplace=True)
df5

,Pclass,Age,SibSp,Parch,Fare
PassengerId,,,,,
892,3,34.5,0,0,7.8292
893,3,47.0,1,0,7.0000
894,2,62.0,0,0,9.6875
895,3,27.0,0,0,8.6625
896,3,22.0,1,1,12.2875
...,...,...,...,...,...
1305,3,28.0,0,0,8.0500
1306,1,39.0,0,0,108.9000
1307,3,38.5,0,0,7.2500


In [ ]:

# 13. membuat prediksi menggunakan model yang sudah dibuat
knn_predictions = knn.predict(df5)
dt_predictions = dt.predict(df5)
rf_predictions = rf.predict(df5)
xgb_predictions = xgb_model.predict(df5)

In [ ]:
# 14. salin data baru yang telah diload dan dibersihkan
combined_data = df5.copy()
# 15. Menambahkan kolom untuk hasil prediksi
combined_data['KNN_Predictions'] = knn_predictions
combined_data['DT_Predictions'] = dt_predictions
combined_data['RF_Predictions'] = rf_predictions
combined_data['XGB_Predictions'] = xgb_predictions
combined_data

,Pclass,Age,SibSp,Parch,Fare,KNN_Predictions,DT_Predictions,RF_Predictions,XGB_Predictions
PassengerId,,,,,,,,,
892,3,34.5,0,0,7.8292,0,0,0,0
893,3,47.0,1,0,7.0000,0,0,0,0
894,2,62.0,0,0,9.6875,0,1,0,0
895,3,27.0,0,0,8.6625,1,1,1,1
896,3,22.0,1,1,12.2875,0,1,1,1
...,...,...,...,...,...,...,...,...,...
1305,3,28.0,0,0,8.0500,0,0,0,0
1306,1,39.0,0,0,108.9000,1,1,1,1
1307,3,38.5,0,0,7.2500,0,0,0,0
